In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras_preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import cv2
import numpy as np
from tqdm import tqdm
import warnings
from keras.models import *
from keras.layers import *
from keras.layers import *
from keras.optimizers import *
from keras import backend as keras
from keras import Model

In [20]:
warnings.filterwarnings('ignore')
plt.rcParams["figure.dpi"] = 120
image_height = 512
image_width = 512
image_depth = 1
epochs=100
batch_size = 32
epoch_index = [i for i in range(1, epochs+1)]

#### Load data

In [21]:
images = np.load(os.path.join("Brain_MRI_data1", "images.npy"), allow_pickle=True)
masks = np.load(os.path.join("Brain_MRI_data1", "masks.npy"), allow_pickle=True)

#### Change all image to grey

In [4]:
image_index = 0
for image in tqdm(images):
    plt.imsave(os.path.join("Brain_MRI_Segment_image", "image_{}.jpg".format(image_index)), image, cmap='gray')
    image_index += 1

mask_index = 0
for mask in tqdm(masks):
    plt.imsave(os.path.join("Brain_MRI_Segment_mask", "mask_{}.jpg".format(mask_index)), mask, cmap='gray')
    mask_index += 1

NameError: name 'images' is not defined

In [ ]:
image_dir_name = "Brain_MRI_Segment_image"
mask_dir_name = "Brain_MRI_Segment_mask"
image_list = list()
mask_list = list()

images_dir = sorted(os.listdir(image_dir_name))
for image in tqdm(images_dir):
    current_image = cv2.imread(os.path.join(image_dir_name, image), cv2.IMREAD_GRAYSCALE)
    current_image = cv2.resize(current_image, (image_height, image_width))
    current_image_180 = cv2.rotate(current_image, cv2.ROTATE_180)
    current_image_r90 = cv2.rotate(current_image, cv2.ROTATE_90_CLOCKWISE)
    current_image_l90 = cv2.rotate(current_image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    image_list.append(current_image)
    image_list.append(current_image_180)
    image_list.append(current_image_r90)
    image_list.append(current_image_l90)



masks_dir = sorted(os.listdir(mask_dir_name))
for mask in tqdm(masks_dir):
    current_mask = cv2.imread(os.path.join(mask_dir_name, mask), cv2.IMREAD_GRAYSCALE)
    current_mask = cv2.resize(current_mask, (image_height, image_width))
    current_mask_180 = cv2.rotate(current_mask, cv2.ROTATE_180)
    current_mask_r90 = cv2.rotate(current_mask, cv2.ROTATE_90_CLOCKWISE)
    current_mask_l90 = cv2.rotate(current_mask, cv2.ROTATE_90_COUNTERCLOCKWISE)
    mask_list.append(current_mask)
    mask_list.append(current_mask_180)
    mask_list.append(current_mask_r90)
    mask_list.append(current_mask_l90)


features = np.array(image_list)/255.0
masks = np.array(mask_list)/255.0


# new_list_of_features = list()
# for arr in tqdm(image_list):
#     new_feature = arr.reshape((image_height, image_width, 1))
#     new_list_of_features.append(new_feature)
#
# new_list_of_features = np.array(new_list_of_features)
#
# new_list_of_masks = list()
# for arr in tqdm(mask_list):
#     new_mask = arr.reshape((image_height, image_width, 1))
#     new_list_of_masks.append(new_mask)
#
# new_list_of_masks = np.array(new_list_of_masks)
features = features.reshape((features.shape[0], features.shape[1], features.shape[2], 1))
masks = masks.reshape((masks.shape[0], masks.shape[1], masks.shape[2], 1))

100%|██████████| 3064/3064 [00:03<00:00, 926.20it/s] 


In [40]:
print("image dimension: {}, label dimension: {}".format(features.shape, masks.shape))

image dimension: (12256, 512, 512, 1), label dimension: (12256, 512, 512, 1)


#### Train, Test, Validation Split

In [41]:
X_train, X_test, y_train, y_test = train_test_split(features, masks, test_size=0.25, random_state=42, shuffle=True)
tf.cast(X_test, tf.float32)
tf.cast(y_test, tf.float32)
train_generator = ImageDataGenerator(validation_split=0.25)
test_generator = ImageDataGenerator()

train_df = train_generator.flow(X_train, y_train, batch_size=batch_size, subset="training")
validation_df = train_generator.flow(X_train, y_train, batch_size=batch_size, subset="validation", seed=42)
test_df = train_generator.flow(X_test, y_test, batch_size=batch_size)

ValueError: Training and validation subsets have different number of classes after the split. If your numpy arrays are sorted by the label, you might want to shuffle them.

#### Define U-Net Model and Loss Function

In [34]:
def dice_coef(y_true, y_pred):
    y_true_f = keras.flatten(y_true)
    y_pred_f = keras.flatten(y_pred)
    intersection = keras.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (keras.sum(y_true_f) + keras.sum(y_pred_f) + 1)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

def unet(input_size=(256,256,1)):
    inputs = Input(input_size)

    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    return Model(inputs=[inputs], outputs=[conv10])


In [35]:
model_Unet = unet(input_size=(image_height,image_width,1))
model_Unet.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef, 'binary_accuracy'])
model_Unet.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 512, 512, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_76 (Conv2D)             (None, 512, 512, 32  320         ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_77 (Conv2D)             (None, 512, 512, 32  9248        ['conv2d_76[0][0]']              
                                )                                                           

In [37]:
unet_history = model_Unet.fit(train_df,
                        epochs=epochs,
                        verbose=1,
                        validation_data=validation_df)

Epoch 1/100


TypeError: in user code:

    File "/Users/wangyaya/opt/anaconda3/envs/CS_HW/lib/python3.8/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/var/folders/_t/5xwcj3s913b6rjc1d1nq30m80000gn/T/ipykernel_31580/2660477695.py", line 8, in dice_coef_loss  *
        return -dice_coef(y_true, y_pred)
    File "/var/folders/_t/5xwcj3s913b6rjc1d1nq30m80000gn/T/ipykernel_31580/2660477695.py", line 4, in dice_coef  *
        intersection = keras.sum(y_true_f * y_pred_f)

    TypeError: Input 'y' of 'Mul' Op has type float32 that does not match type uint8 of argument 'x'.
